In [4]:
import pandas as pd

# Replace 'data.xlsx' with your Excel file's name and path if it's located in a different directory
file_path = 'data.xlsx'

# Load the data from the Excel file
data = pd.read_excel(file_path)
column_to_remove = 'Unnamed: 0'
data.drop(column_to_remove, axis=1, inplace=True)
data['Date'] = pd.to_datetime(data['Date'], format='%b %Y')
data.set_index('Date', inplace=True)
data = data.pct_change().dropna()*100
data = data.reset_index()
data

# Replace 'data.xlsx' with your Excel file's name and path if it's located in a different directory
file_path = 'OutProcess.xlsx'

# Load the data from the Excel file
cpi = pd.read_excel(file_path)

cpi.head()

merged_df = pd.merge(data, cpi, on='Date', how='inner')
merged_df.set_index('Date', inplace=True)
merged_df.head()
# We keep the the most correlated variables
columns_to_keep = ['lag_Food', 'lag_Clothing/Shoes', 'Household/Furnishing/Equipment','lag_Household/Furnishing/Equipment','GDP','CPI','Unemployment','Wages',	'Balance of Payments',	'Consumption Spending',	
                   'Inventory Investment',	'Rate']

# Create a new DataFrame with only the desired columns
df_filtered = merged_df[columns_to_keep]
new_column_names = {
    'Balance of Payments': 'NetExports',
    'Consumption Spending': 'Consumptions',
    'Inventory Investment': 'Investments',
    'lag_Food' :'LFood',
    'lag_Clothing/Shoes': 'LClothShoes'	,
    'Household/Furnishing/Equipment':'HouseFurnEquip',
    'lag_Household/Furnishing/Equipment':'LHouseFurnEquip'
}

df_filtered.rename(columns=new_column_names, inplace=True)
df_filtered.index = df_filtered.index.strftime("%Y-%m")
df_filtered['CPI']=((1+df_filtered['CPI']/100)**12-1)*100
df_filtered
df_filtered = df_filtered.reset_index()
df_filtered

# Set the index back to the date index
df_filtered.set_index('Date', inplace=True)


df_filtered

FileNotFoundError: [Errno 2] No such file or directory: 'data.xlsx'

In [ ]:
new_column_names = {
    'Balance of Payments': 'NetExports',
    'Consumption Spending': 'Consumptions',
    'Inventory Investment': 'Investments',
    'lag_Food' :'LFood',
    'lag_Clothing/Shoes': 'LClothShoes'	,
    'Household/Furnishing/Equipment':'HouseFurnEquip',
    'lag_Household/Furnishing/Equipment':'LHouseFurnEquip'
}

df_filtered.rename(columns=new_column_names, inplace=True)
df_filtered.index = df_filtered.index.strftime("%Y-%m")
df_filtered['CPI']=((1+df_filtered['CPI']/100)**12-1)*100
df_filtered
df_filtered = df_filtered.reset_index()
df_filtered

# Set the index back to the date index
df_filtered.set_index('Date', inplace=True)

#CPI=((1+df_filtered['CPI']/100)**12-1)*100
#df_filtered.drop(columns=['CPI'], inplace=True)
#df_filtered
#df_filtered['CPI'] =CPI
#df_filtered
df_filtered

AttributeError: 'Index' object has no attribute 'strftime'

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
# initialize newdf
x=0
ten_factor_model= smf.ols(
        formula='CPI ~ GDP + Unemployment + Wages + Rate + CS +HouseFurnEquip+LHouseFurnEquip+LClothShoes+LFood+II+BOP',
        data=df_filtered[x:x+10]
    ).fit()

intercept, b1, b2, b3,b4,b5,b6,b7,b8,b9,b10,b11 = ten_factor_model.params
array=ten_factor_model.params
array


PatsyError: Error evaluating factor: NameError: name 'II' is not defined
    CPI ~ GDP + Unemployment + Wages + Rate + CS +HouseFurnEquip+LHouseFurnEquip+LClothShoes+LFood+II+BOP
                                                                                                   ^^

In [ ]:
NewAr = pd.DataFrame([array])
CoefLogger=NewAr
CoefLogger


,Intercept,GDP,Unemployment,Wages,Rate,CS,HouseFurnEquip,LHouseFurnEquip,LClothShoes,LFood,II,BOP
0,4.132664,-8.890592,-5.150327,-11.552955,7.663965,1.554781,46.416058,8.15981,0.289015,7.515731,-0.259318,1.065025


In [ ]:
CoefLogger = CoefLogger.drop(CoefLogger.index[0])
CoefLogger

,Intercept,GDP,Unemployment,Wages,Rate,CS,HouseFurnEquip,LHouseFurnEquip,LClothShoes,LFood,II,BOP


In [5]:
for x in range(0,120):
    ten_factor_model= smf.ols(
        formula='CPI ~ GDP + Unemployment + Wages + Rate + CS +HouseFurnEquip+LHouseFurnEquip+LClothShoes+LFood+II+BOP',
        data=df_filtered[x:x+10]
    ).fit()

    #intercept, b1, b2, b3,b4,b5,b6,b7,b8,b9,b10 = ten_factor_model.params


    array=ten_factor_model.params
    NewAr = pd.DataFrame([array])
    CoefLogger = CoefLogger.append(NewAr, ignore_index=True)
CoefLogger

PatsyError: Error evaluating factor: NameError: name 'II' is not defined
    CPI ~ GDP + Unemployment + Wages + Rate + CS +HouseFurnEquip+LHouseFurnEquip+LClothShoes+LFood+II+BOP
                                                                                                   ^^

In [ ]:
CoefLogger.index=df_filtered.index
CoefLogger['CPI'] = df_filtered['CPI']
CoefLogger.tail(20)

,Intercept,GDP,Unemployment,Wages,Rate,CS,HouseFurnEquip,LHouseFurnEquip,LClothShoes,LFood,II,BOP,CPI
Date,,,,,,,,,,,,,
2021-06,1.142559,-9.029847,-1.259945,3.913251,0.107453,-0.633693,6.704744,11.727055,-8.050200,-5.678475,0.013107,0.010589,3.457877
2021-07,7.472706,-6.194533,-0.963347,-7.865866,-0.044606,-10.176762,25.482267,13.607412,-19.783054,3.570161,-0.017973,0.021445,7.911042
2021-08,6.537618,6.997236,0.394727,-7.594043,-0.065940,10.425139,12.735484,3.765022,-16.952273,-10.343199,-0.001815,0.017875,2.559408
2021-09,-1.762037,5.695436,-0.366394,1.626831,-0.005539,9.661291,22.960781,3.905129,-14.463665,-16.930203,0.005349,0.002486,2.553961
2021-10,0.449603,2.491680,0.706828,-8.381403,-0.035162,13.511977,17.205328,8.962759,-7.937683,-13.119417,0.018657,0.001635,8.728346
2021-11,-1.466175,-7.077451,0.480583,-9.442814,0.011427,6.505273,17.781550,9.747935,-4.844714,-9.261229,0.168753,0.001200,2.530623
2021-12,5.185627,1.163360,-0.958532,2.922692,0.074999,-15.538899,16.016416,5.373442,-13.881606,-7.305773,0.285240,0.001056,-1.651717
2022-01,-8.438007,0.409886,1.668684,-3.801436,0.023872,-3.868099,5.536140,0.701563,30.203924,20.464560,0.261280,-0.002739,11.387759
2022-02,-10.894879,-1.538288,-0.028216,-9.522793,0.006300,9.770138,6.336455,3.729776,5.124150,12.634542,-0.121966,-0.000052,13.116327


In [6]:
ten_factor_data=CoefLogger.tail(40)
num_rows = len(ten_factor_data)
import numpy as np
import matplotlib.pyplot as plt

print("Number of rows:", num_rows)
time=num_rows

categories = ['GDP', 'Unemployment', 'Wages', 'Rate','Consumption','Household/Furnishing/Equipment_Inflation','Lagged_Household/Furnishing/Equipment_Inflation','Lagged_Clothing_Inflation','Lagged_Food_Inflation','Investment','BOP',]
time_periods = ten_factor_data.index[:time].tolist()
data = np.array([ten_factor_data['GDP'].tolist()[:time],  # mkt
                 ten_factor_data['Unemployment'].tolist()[:time],   # smb'CPI ~ GDP + Unemployment + Wages + BOP + Rate + CS + II+HouseFurnEquip+LClothShoes+LFood'
                 ten_factor_data['Wages'].tolist()[:time],
                 ten_factor_data['Rate'].tolist()[:time],
                 ten_factor_data['CS'].tolist()[:time],   # cma
                 ten_factor_data['HouseFurnEquip'].tolist()[:time],
                 ten_factor_data['LHouseFurnEquip'].tolist()[:time],
                 ten_factor_data['LClothShoes'].tolist()[:time],
                 ten_factor_data['LFood'].tolist()[:time],
                 ten_factor_data['II'].tolist()[:time],
                 ten_factor_data['BOP'].tolist()[:time],
                 ])  

# Create stacked bar chart
fig, ax = plt.subplots()

# Plotting each category's bar
bottom = np.zeros(len(time_periods))
for i, category in enumerate(categories):
    ax.bar(time_periods, data[i], bottom=bottom, label=category)
    bottom += data[i]

# Customize the plot
ax.set_xlabel('Time')
ax.set_ylabel('Inflation')
ax.set_title('CPI Factor VS Actual CPI')
ax.legend()

# Calculate trend line
x = np.arange(len(time_periods))
y=ten_factor_data['CPI'].tolist()[:time]

# Plot the trend line
ax.plot(x, y, color='black', label='Actual CPI')
ax.legend()

# Set the legend below the graph
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=4)

# Set the x-axis tick labels to display horizontally
plt.xticks(rotation=90)
plt.xticks(fontsize=10)


# Display the chart
plt.show()


NameError: name 'CoefLogger' is not defined